# ポケモンカードデッキ プロキシ印刷用PDF化

**作成者：tanadaaa**  
X:@tanadaaa29

> **注意**
> - V-UNIONカードは未対応の場合があります。
> - Pythonやライブラリのバージョンを固定していません。動作しなくなった場合はご連絡ください。

## 使い方
1. 最下部のセルの方で、`deck_code = "Syp2UX-QiBj4c-pSyypM"` のようにデッキコードを設定します  
2. すべてのセルを上から順に実行してください  
3. 実行完了後、左側のファイルビューの`deck`フォルダ内にPDFが出力されます。自動でダウンロードしたい場合は、最下部のセル内の指示に従って#を外してください。


In [ ]:
!uv pip install reportlab

In [ ]:
import os

os.makedirs('decks', exist_ok=True)

In [ ]:
import sys
import re
import requests
import time
import random
from tqdm.auto import tqdm
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import mm
from reportlab.lib.utils import ImageReader
from google.colab import files

BASE_URL = "https://www.pokemon-card.com"

def scrape_deck_with_images(deck_code: str):
    """
    deck_code → [
      {"id":"45199","count":3,"image_url":"...045199_P_TAKERURAIKOEX.jpg"},
      …
    ]
    デッキコードエラー時は ValueError を投げます。
    """
    url = f"{BASE_URL}/deck/result.html/deckID/{deck_code}/"
    resp = requests.get(url)
    resp.encoding = resp.apparent_encoding
    soup = BeautifulSoup(resp.text, "html.parser")

    # デッキコードエラー判定
    h1 = soup.find("h1", class_="Heading1")
    if h1 and "デッキコードエラー" in h1.text:
        raise ValueError("デッキコードエラー")

    # inputArea からカードID＋枚数 を取得
    form = soup.find("form", id="inputArea")
    if not form:
        raise RuntimeError("inputArea が見つかりませんでした。")

    deck_fields = [
        "deck_pke", "deck_gds", "deck_tool", "deck_tech",
        "deck_sup", "deck_sta", "deck_ene", "deck_ajs"
    ]
    cards = []
    for field in deck_fields:
        inp = form.find("input", id=field)
        if not inp or not inp.get("value"):
            continue
        for token in inp["value"].split("-"):
            parts = token.split("_")
            card_id, count = parts[0], int(parts[1])
            cards.append({"id": card_id, "count": count})

    # JS 部分から画像URL を抜き出し
    all_scripts = "".join(s.string or "" for s in soup.find_all("script"))
    pattern = re.compile(r"PCGDECK\.searchItemCardPict\[(\d+)\]='([^']+)';")
    mapping = {
        m.group(1): urljoin(BASE_URL, m.group(2))
        for m in pattern.finditer(all_scripts)
    }

    # 各エントリに image_url を付与
    for c in cards:
        c["image_url"] = mapping.get(
            c["id"],
            urljoin(BASE_URL, "/assets/images/noimage/poke_ura.jpg")
        )
    return cards

def generate_deck_pdf(deck_code: str, output_pdf: str):
    # スクレイプ
    cards = scrape_deck_with_images(deck_code)

    # 画像を一度だけキャッシュ
    session = requests.Session()
    # User-Agentを設定
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    })

    img_cache = {}
    for c in tqdm(cards, desc="Downloading images"):
        url = c["image_url"]
        if url not in img_cache:
            time.sleep(random.uniform(0.1, 0.3))
            r = session.get(url, stream=True, timeout=10)
            r.raise_for_status()
            img_cache[url] = ImageReader(r.raw)

    # 平坦化して60枚分のリストを作成
    flattened = []
    for c in cards:
        flattened += [c["image_url"]] * c["count"]

    # PDF 作成
    c = canvas.Canvas(output_pdf, pagesize=A4)
    margin_x = 10.5 * mm   # 左右マージン
    margin_y = 16.5 * mm   # 上下マージン
    card_w   = 63   * mm
    card_h   = 88   * mm
    per_page = 9

    for idx, img_url in tqdm(enumerate(flattened), desc="Generating PDF", total=len(flattened)):
        if idx % per_page == 0 and idx > 0:
            c.showPage()
        pos = idx % per_page
        col = pos % 3
        row = pos // 3
        x = margin_x + col * card_w
        y = margin_y + (2 - row) * card_h
        c.drawImage(img_cache[img_url], x, y, width=card_w, height=card_h)

    c.save()


## デッキコードを指定

In [ ]:
# この" "で囲まれた部分の文字列を書き換える
deck_code = "YYx4D8-ou796p-xY88cc"

In [ ]:
output_pdf = f"decks/{deck_code}.pdf"

try:
    generate_deck_pdf(deck_code, output_pdf)
    print("PDF を生成しました:", output_pdf)
    #files.download(output_pdf) # 勝手にダウンロードされる設定をオンにしたければ、先頭の#を削除してください。
except Exception as e:
    print("エラーが発生しました:", e)
